In [70]:
from sklearn.model_selection import KFold
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from scipy import stats

def rmse(predictions, targets):
    count = 0
    k = []
    for x in predictions:
        temp = np.sqrt(((predictions[count] - targets[count]) ** 2)/2)
        count+=1
        k.append(temp)
    return k


df = pd.read_csv('drop_out.csv')
# df = pd.read_csv('fill.csv')


# print(X.columns)
Vet = df['PCV_target']
# X = df[['Weight', 'PCV', 'PCV\ndonor', 'Volume', 'WBC', 'PLT\n______', 'HGB', 'RBC', 'MCV', 'MCHC', 'MCH',
#         'SEGS', 'LYMPH', 'MONO','RDW']]
X = df[['Weight', 'PCV', 'PCV\ndonor', 'Volume']]
# X = df[['Weight', 'PCV', 'PCV\ndonor', 'MCV', 'MCHC', 'MCH']]
y = df['PCV_afterdonation']
column_name = X.columns

# # convert to numeric
# X = X.apply(pd.to_numeric, errors='coerce')
# y = y.apply(pd.to_numeric, errors='coerce')
# Vet = Vet.apply(pd.to_numeric, errors='coerce')



# # fill with mean
# X = X.fillna(X.mean())
# y = y.fillna(y.mean())
# Vet = Vet.fillna(Vet.mean())


X = np.array(X)
y = np.array(y)
Vet = np.array(Vet)


scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)


kf = KFold(n_splits=10)
kf.get_n_splits(X)

RMSE=[]

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lm = LinearRegression()
    lm.fit(X_train,y_train)
    predictions = lm.predict(X_test)
    temp_RMSE = np.sqrt(metrics.mean_squared_error(y_test, predictions))
    RMSE.append(temp_RMSE)
#     print(temp_RMSE)


    
print ('Average RMSE 10 fold cross validation : %f'%np.mean(RMSE))


RMSE=[]
train_RMSE=[]
predict_for_p=[]
# print("leave one out")

from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
loo.get_n_splits(X)

# print(loo)
coef_list = []
for train_index, test_index in loo.split(X):
#    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#    print(X_train, X_test, y_train, y_test)

    lm = LinearRegression()
    lm.fit(X_train,y_train)
    
#     for train accuracy
    xxx = lm.predict(X_train)
    temp_train_RMSE = np.sqrt(metrics.mean_squared_error(y_train, xxx))
    train_RMSE.append(temp_train_RMSE)
#     asdfafdafd


    predictions = lm.predict(X_test)
    temp_RMSE = np.sqrt(metrics.mean_squared_error(y_test, predictions))
    RMSE.append(temp_RMSE)
    predict_for_p.append(predictions)
#     print(lm.coef_)
    coef_list.append(lm.coef_)
#     print(temp_RMSE)
print ('Average RMSE Leave one out : %f ± %f'%(np.mean(RMSE), np.std(RMSE)))
print ('average train RMSE  : %f ± %f'%(np.mean(train_RMSE), np.std(train_RMSE)))

print("********")
rmse_vet = rmse(Vet,y)
# print(rmse_vet)
# print(stats.ttest_rel(RMSE,rmse_vet).pvalue)
p_value = stats.ttest_rel(RMSE,rmse_vet).pvalue
print('P value is %f'%p_value)


print ('RMSE Vet : %f'%np.mean(np.sqrt(metrics.mean_squared_error(Vet, y))))
print ('Average Coef')
coef_list = np.matrix(coef_list)
coef_list = np.absolute(coef_list)
# print(column_name)
x = coef_list.mean(0)
# print (x)
array_x = np.array(x)
# print(array_x)

coeff_df = pd.DataFrame(array_x[0],column_name,columns=['Coef'])
coeff_df = coeff_df.sort_values(by=['Coef'], ascending=False)
print(coeff_df)

Average RMSE 10 fold cross validation : 4.884423
Average RMSE Leave one out : 3.918339 ± 2.880059
average train RMSE  : 4.248371 ± 0.072313
********
P value is 0.111673
RMSE Vet : 5.713195
Average Coef
                Coef
PCV         5.907297
Weight      3.413506
Volume      0.873684
PCV\ndonor  0.335891
